In [17]:
import time
import requests
import datetime
import pandas as pd

from util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'
path_data = './data'
path_csv = path_data + '/csv'
file_name_out_r = path_csv + '/covid19-' + level + '.csv'
file_name_pop_r = path_csv + '/popolazione-regioni.csv'
file_name_icu_r = path_csv + '/terapie-intensive.csv'

In [18]:
# Define today, yesterday and day before yesterday dates
date_tday = datetime.date.today()
date_yday = Functions.get_previous_date(date_tday)
date_bf_yday = Functions.get_previous_date(date_yday)
date_tday, date_yday, date_bf_yday

(datetime.date(2020, 10, 25),
 datetime.date(2020, 10, 24),
 datetime.date(2020, 10, 23))

In [19]:
# Today url for regions file
url_tday = Functions.get_url_date(level, date_tday)
# Yesterday url for regions file
url_yday = Functions.get_url_date(level, date_yday)
# Day before yesterday url for regions file
url_bf_yday = Functions.get_url_date(level, date_bf_yday)

url_tday, url_yday, url_bf_yday

('https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni-20201025.csv',
 'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni-20201024.csv',
 'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni-20201023.csv')

In [20]:
if False:
    while True:
        status_code = requests.get(url_tday).status_code
        if status_code == 200:
            print('Presenti dati odierni')
            break
        print('Status code: {}. Retrying...'.format(status_code))
        time.sleep(20)

In [21]:
# Get csv files
try:
    # today and yesterday if today present
    df_raw_r_1 = pd.read_csv(url_tday)
    print('Dati odierni')
    df_raw_r_0 = pd.read_csv(url_yday)
    date_last = date_tday
except Exception as e:
    # yesterday and day before yesterday otherwise
    df_raw_r_1 = pd.read_csv(url_yday)
    df_raw_r_0 = pd.read_csv(url_bf_yday)
    date_last = date_yday
    print('Dati di ieri')

Dati odierni


In [22]:
# State S
df_raw_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-10-25T17:00:00,ITA,1,Piemonte,45.073274,7.680687,1601,94,1695,18182,...,2034,2287,31399,4259,28556,26979,55535,944133,566796,NaN
1,2020-10-25T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,63,2,65,1116,...,88,97,1191,153,2265,260,2525,37826,24256,NaN
2,2020-10-25T17:00:00,ITA,3,Lombardia,45.466794,9.190347,2326,231,2557,45426,...,5298,5762,89145,17235,127457,26906,154363,2715715,1698094,NaN
3,2020-10-25T17:00:00,ITA,5,Veneto,45.434905,12.338452,585,71,656,16316,...,1351,1468,25036,2329,24005,20332,44337,2238353,871568,NaN
4,2020-10-25T17:00:00,ITA,6,Friuli Venezia Giulia,45.649435,13.768136,112,23,135,2892,...,251,334,4760,374,7375,786,8161,508057,219485,NaN
5,2020-10-25T17:00:00,ITA,7,Liguria,44.411493,8.932699,696,41,737,5299,...,276,657,15324,1691,16455,6596,23051,409770,214109,NaN
6,2020-10-25T17:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,757,88,845,13983,...,1186,1192,27345,4561,36934,9800,46734,1476451,804679,NaN
7,2020-10-25T17:00:00,ITA,9,Toscana,43.769231,11.255889,714,111,825,16495,...,1632,1863,12708,1262,24763,6527,31290,1000835,671386,NaN
8,2020-10-25T17:00:00,ITA,10,Umbria,43.106758,12.388247,215,29,244,4151,...,374,463,2795,105,2575,4720,7295,277303,162084,NaN
9,2020-10-25T17:00:00,ITA,11,Marche,43.616760,13.518875,178,23,201,3525,...,427,521,6711,1003,11440,0,11440,303279,177287,NaN


In [23]:
# State S - 1
df_raw_r_0.sort_values(by='codice_regione').reset_index(drop=True)

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-10-24T17:00:00,ITA,1,Piemonte,45.073274,7.680687,1483,88,1571,16272,...,1381,1548,31157,4248,27787,25461,53248,931476,556121,NaN
1,2020-10-24T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,58,2,60,1033,...,92,112,1183,152,2177,251,2428,37525,24087,NaN
2,2020-10-24T17:00:00,ITA,3,Lombardia,45.466794,9.190347,2153,213,2366,40319,...,4735,4956,88706,17210,122847,25754,148601,2680430,1675053,NaN
3,2020-10-24T17:00:00,ITA,5,Veneto,45.434905,12.338452,548,68,616,15005,...,1587,1729,24931,2317,23881,18988,42869,2226292,866907,NaN
4,2020-10-24T17:00:00,ITA,6,Friuli Venezia Giulia,45.649435,13.768136,100,16,116,2660,...,299,412,4679,372,7112,715,7827,504257,218014,NaN
5,2020-10-24T17:00:00,ITA,7,Liguria,44.411493,8.932699,641,38,679,5081,...,291,1035,14948,1686,15958,6436,22394,405575,212171,NaN
6,2020-10-24T17:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,723,89,812,12830,...,1128,1180,27343,4557,36378,9164,45542,1466807,799074,In seguito a verifica sui dati comunicati nei ...
7,2020-10-24T17:00:00,ITA,9,Toscana,43.769231,11.255889,640,87,727,14961,...,1314,1526,12491,1248,23194,6233,29427,987024,662605,NaN
8,2020-10-24T17:00:00,ITA,10,Umbria,43.106758,12.388247,203,25,228,3793,...,391,525,2709,102,2412,4420,6832,273906,160255,NaN
9,2020-10-24T17:00:00,ITA,11,Marche,43.616760,13.518875,178,23,201,3098,...,250,274,6619,1001,10919,0,10919,299892,174895,NaN


In [24]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-10-25T17:00:00,ITA,1,Piemonte,45.073274,7.680687,1601,94,1695,18182,...,2034,2287,31399,4259,28556,26979,55535,944133,566796,NaN
1,2020-10-25T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,63,2,65,1116,...,88,97,1191,153,2265,260,2525,37826,24256,NaN
2,2020-10-25T17:00:00,ITA,3,Lombardia,45.466794,9.190347,2326,231,2557,45426,...,5298,5762,89145,17235,127457,26906,154363,2715715,1698094,NaN
3,2020-10-25T17:00:00,ITA,4,Trentino-Alto Adige,46.382635,11.204676,219,15,234,4285,...,407,443,8986,721,10848,3378,14226,493814,224633,NaN
4,2020-10-25T17:00:00,ITA,5,Veneto,45.434905,12.338452,585,71,656,16316,...,1351,1468,25036,2329,24005,20332,44337,2238353,871568,NaN
5,2020-10-25T17:00:00,ITA,6,Friuli Venezia Giulia,45.649435,13.768136,112,23,135,2892,...,251,334,4760,374,7375,786,8161,508057,219485,NaN
6,2020-10-25T17:00:00,ITA,7,Liguria,44.411493,8.932699,696,41,737,5299,...,276,657,15324,1691,16455,6596,23051,409770,214109,NaN
7,2020-10-25T17:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,757,88,845,13983,...,1186,1192,27345,4561,36934,9800,46734,1476451,804679,NaN
8,2020-10-25T17:00:00,ITA,9,Toscana,43.769231,11.255889,714,111,825,16495,...,1632,1863,12708,1262,24763,6527,31290,1000835,671386,NaN
9,2020-10-25T17:00:00,ITA,10,Umbria,43.106758,12.388247,215,29,244,4151,...,374,463,2795,105,2575,4720,7295,277303,162084,NaN


In [25]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,11,2020-10-24T17:00:00,ITA,1,Piemonte,45.073274,7.680687,1483,88,1571,...,1381,1548,31157,4248,27787,25461,53248,931476,556121,NaN
1,17,2020-10-24T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,58,2,60,...,92,112,1183,152,2177,251,2428,37525,24087,NaN
2,8,2020-10-24T17:00:00,ITA,3,Lombardia,45.466794,9.190347,2153,213,2366,...,4735,4956,88706,17210,122847,25754,148601,2680430,1675053,NaN
3,19,2020-10-24T17:00:00,ITA,4,Trentino-Alto Adige,46.382635,11.204676,192,16,208,...,401,439,8951,720,10446,3337,13783,489234,222970,NaN
4,18,2020-10-24T17:00:00,ITA,5,Veneto,45.434905,12.338452,548,68,616,...,1587,1729,24931,2317,23881,18988,42869,2226292,866907,NaN
5,5,2020-10-24T17:00:00,ITA,6,Friuli Venezia Giulia,45.649435,13.768136,100,16,116,...,299,412,4679,372,7112,715,7827,504257,218014,NaN
6,7,2020-10-24T17:00:00,ITA,7,Liguria,44.411493,8.932699,641,38,679,...,291,1035,14948,1686,15958,6436,22394,405575,212171,NaN
7,4,2020-10-24T17:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,723,89,812,...,1128,1180,27343,4557,36378,9164,45542,1466807,799074,In seguito a verifica sui dati comunicati nei ...
8,15,2020-10-24T17:00:00,ITA,9,Toscana,43.769231,11.255889,640,87,727,...,1314,1526,12491,1248,23194,6233,29427,987024,662605,NaN
9,16,2020-10-24T17:00:00,ITA,10,Umbria,43.106758,12.388247,203,25,228,...,391,525,2709,102,2412,4420,6832,273906,160255,NaN


In [26]:
# Print raw columns
# print(df_r_1.columns, df_r_0.columns)
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note']
df_rf_1 = df_r_1.drop(cols_to_drop, axis=1).sort_values(by='codice_regione').reset_index(drop=True)
df_rf_0 = df_r_0.drop(cols_to_drop, axis=1).sort_values(by='codice_regione').reset_index(drop=True)
# df_rf_1.columns, df_rf_0.columns

In [27]:
df_rf_0 = df_rf_0.sort_values(by='codice_regione').reset_index(drop=True)
df_rf_0;

In [28]:
df_r = df_rf_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(file_name_pop_r)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione
0,2020-10-25T17:00:00,1,Piemonte,1601,94,1695,18182,19877,2034,2287,31399,4259,28556,26979,55535,944133,566796,4341375
1,2020-10-25T17:00:00,2,Valle d'Aosta,63,2,65,1116,1181,88,97,1191,153,2265,260,2525,37826,24256,125501
2,2020-10-25T17:00:00,3,Lombardia,2326,231,2557,45426,47983,5298,5762,89145,17235,127457,26906,154363,2715715,1698094,10103969
3,2020-10-25T17:00:00,4,Trentino-Alto Adige,219,15,234,4285,4519,407,443,8986,721,10848,3378,14226,493814,224633,1074819
4,2020-10-25T17:00:00,5,Veneto,585,71,656,16316,16972,1351,1468,25036,2329,24005,20332,44337,2238353,871568,4907704
5,2020-10-25T17:00:00,6,Friuli Venezia Giulia,112,23,135,2892,3027,251,334,4760,374,7375,786,8161,508057,219485,1211357
6,2020-10-25T17:00:00,7,Liguria,696,41,737,5299,6036,276,657,15324,1691,16455,6596,23051,409770,214109,1543127
7,2020-10-25T17:00:00,8,Emilia-Romagna,757,88,845,13983,14828,1186,1192,27345,4561,36934,9800,46734,1476451,804679,4467118
8,2020-10-25T17:00:00,9,Toscana,714,111,825,16495,17320,1632,1863,12708,1262,24763,6527,31290,1000835,671386,3722729
9,2020-10-25T17:00:00,10,Umbria,215,29,244,4151,4395,374,463,2795,105,2575,4720,7295,277303,162084,880285


In [29]:
# Merge with icu data
df_icu_r = pd.read_csv(file_name_icu_r)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione,totale_posti_terapia_intensiva
0,2020-10-25T17:00:00,1,Piemonte,1601,94,1695,18182,19877,2034,2287,31399,4259,28556,26979,55535,944133,566796,4341375,367
1,2020-10-25T17:00:00,2,Valle d'Aosta,63,2,65,1116,1181,88,97,1191,153,2265,260,2525,37826,24256,125501,20
2,2020-10-25T17:00:00,3,Lombardia,2326,231,2557,45426,47983,5298,5762,89145,17235,127457,26906,154363,2715715,1698094,10103969,983
3,2020-10-25T17:00:00,4,Trentino-Alto Adige,219,15,234,4285,4519,407,443,8986,721,10848,3378,14226,493814,224633,1074819,106
4,2020-10-25T17:00:00,5,Veneto,585,71,656,16316,16972,1351,1468,25036,2329,24005,20332,44337,2238353,871568,4907704,825
5,2020-10-25T17:00:00,6,Friuli Venezia Giulia,112,23,135,2892,3027,251,334,4760,374,7375,786,8161,508057,219485,1211357,175
6,2020-10-25T17:00:00,7,Liguria,696,41,737,5299,6036,276,657,15324,1691,16455,6596,23051,409770,214109,1543127,209
7,2020-10-25T17:00:00,8,Emilia-Romagna,757,88,845,13983,14828,1186,1192,27345,4561,36934,9800,46734,1476451,804679,4467118,516
8,2020-10-25T17:00:00,9,Toscana,714,111,825,16495,17320,1632,1863,12708,1262,24763,6527,31290,1000835,671386,3722729,415
9,2020-10-25T17:00:00,10,Umbria,215,29,244,4151,4395,374,463,2795,105,2575,4720,7295,277303,162084,880285,70


In [30]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [31]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_rf_1['ricoverati_con_sintomi'] - df_rf_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_rf_1['terapia_intensiva'] - df_rf_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_rf_1['totale_ospedalizzati'] - df_rf_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_rf_1['isolamento_domiciliare'] - df_rf_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_rf_1['nuovi_positivi'] - df_rf_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_rf_1['tamponi'] - df_rf_0['tamponi']
df_r['nuovi_casi_testati'] = df_rf_1['casi_testati'] - df_rf_0['casi_testati']
df_r['nuovi_deceduti'] = df_rf_1['deceduti'] - df_rf_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_rf_1['dimessi_guariti'] - df_rf_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2020-10-25T17:00:00,3,Lombardia,2326,231,2557,45426,47983,5298,5762,...,23.50,5107,806,35285,23041,25,439,16.33,0.05703,1.53
1,2020-10-25T17:00:00,15,Campania,1151,113,1264,27456,28720,2417,2590,...,26.46,2379,872,16906,7435,0,173,15.32,0.04476,0.67
2,2020-10-25T17:00:00,1,Piemonte,1601,94,1695,18182,19877,2034,2287,...,25.61,1910,739,12657,10675,11,242,18.07,0.05268,1.28
3,2020-10-25T17:00:00,9,Toscana,714,111,825,16495,17320,1632,1863,...,26.75,1534,337,13811,8781,14,217,13.49,0.05004,0.84
4,2020-10-25T17:00:00,12,Lazio,1476,146,1622,21193,22815,1410,1541,...,19.54,1367,-146,21832,16170,10,121,7.06,0.02627,0.58
5,2020-10-25T17:00:00,5,Veneto,585,71,656,16316,16972,1351,1468,...,8.61,1311,-261,12061,4661,12,105,12.17,0.02991,0.90
6,2020-10-25T17:00:00,8,Emilia-Romagna,757,88,845,13983,14828,1186,1192,...,17.05,1153,12,9644,5605,4,2,12.36,0.02668,1.05
7,2020-10-25T17:00:00,19,Sicilia,642,95,737,9818,10555,666,695,...,17.66,625,-191,5193,3834,11,18,13.38,0.01399,0.34
8,2020-10-25T17:00:00,7,Liguria,696,41,737,5299,6036,276,657,...,19.62,218,-378,4195,1938,5,376,15.66,0.04258,1.49
9,2020-10-25T17:00:00,11,Marche,178,23,201,3525,3726,427,521,...,18.11,427,247,3387,2392,2,92,15.38,0.03431,0.75


In [32]:
# Write pre-processed data
df_r.to_csv(file_name_out_r, index=False)